# Simple Neural Network

## Dataset
This dataset usage for Kaggle InClass Competition Telecom Churn Analytics.

The dataset used of this coding,you can downloading in this link.

https://www.kaggle.com/c/churn-analytics-bda

###### Import Libraries

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sc
import warnings
from sklearn.model_selection import train_test_split
from sklearn import cross_validation,metrics
from sklearn.metrics import classification_report, confusion_matrix,roc_curve,auc
from IPython.display import Image
from IPython.core.display import HTML 
from keras.models import Sequential
from keras.layers import Dense
from keras import layers
from sklearn.feature_extraction.text import CountVectorizer

warnings.simplefilter("ignore")
%matplotlib inline

/home/leandro/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/leandro/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
train = pd.read_csv('Datasets/churnTrain.csv')
test = pd.read_csv('Datasets/churnTest.csv')

In [4]:
# Adiciona coluna Churn no dataset de teste para poder fazer merge entre os dois.
test['Churn'] = 'TESTE'
dataframes = [train, test]
churnData = pd.concat(dataframes) 

#### Feature Engineering

N/A treatment, applied mean values in N/A columns.

In [5]:
churnData['Voice_Mail_Plan'].fillna(' no',inplace=True)
churnData['International_Plan'].fillna(' no',inplace=True)
churnData['Account_Length'].fillna(churnData['Account_Length'].mean(),inplace=True)
churnData['Total_Night_Minutes'].fillna(churnData['Total_Night_Minutes'].mean(),inplace=True)
churnData['Total_Intl_Calls'].fillna(churnData['Total_Intl_Calls'].mean(),inplace=True)
churnData['Total_Night_Charge'].fillna(churnData['Total_Night_Charge'].mean(),inplace=True)
churnData['Total_Night_Calls'].fillna(churnData['Total_Night_Calls'].mean(),inplace=True)
churnData['Total_Night_Minutes'].fillna(churnData['Total_Night_Minutes'].mean(),inplace=True)
churnData['Total_Eve_Charge'].fillna(churnData['Total_Eve_Charge'].mean(),inplace=True)
churnData['Total_Eve_Calls'].fillna(churnData['Total_Eve_Calls'].mean(),inplace=True)
churnData['Total_Eve_Minutes'].fillna(churnData['Total_Eve_Minutes'].mean(),inplace=True)
churnData['Total_Day_charge'].fillna(churnData['Total_Day_charge'].mean(),inplace=True)
churnData['Total_Day_Calls'].fillna(churnData['Total_Day_Calls'].mean(),inplace=True)
churnData['Total_Day_minutes'].fillna(churnData['Total_Day_minutes'].mean(),inplace=True)

In [6]:
churnData['Churn'] = churnData['Churn'].astype('category')
churnData['State'] = churnData['State'].astype('category')
churnData['Voice_Mail_Plan'] = churnData['Voice_Mail_Plan'].astype('category')
churnData['International_Plan'] = churnData['International_Plan'].astype('category')

churnData['Churn'] = churnData['Churn'].cat.codes
churnData['State_cat'] = churnData['State'].cat.codes

##### One Hot Enconding - OHE

In [7]:
cols_to_transform = ['International_Plan','Voice_Mail_Plan']
churnData = pd.get_dummies(churnData, columns=cols_to_transform)

##### Train and test Split

In [8]:
## Train and test separated again
churnData = churnData[['Account_Length','Area_Code','Phone_No','No_Vmail_Messages','Total_Day_minutes','Total_Day_Calls',
                      'Total_Day_charge','Total_Night_Calls','Total_Night_Charge','Total_Intl_Minutes','Total_Intl_Calls',
                      'Total_Intl_Charge','No_CS_Calls','State_cat','Voice_Mail_Plan_ no','Voice_Mail_Plan_ yes',
                       'International_Plan_ no','International_Plan_ yes','Churn']]


churnTrain = churnData.query('Churn!=2')
churnTest = churnData.query('Churn ==2')
churnTest.drop('Churn', axis=1)



Y = pd.factorize(churnTrain['Churn'].values)[0].reshape(-1,1)
X = churnTrain.drop('Churn',axis=1)

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.33, random_state=42)

In [9]:
churnData.columns = ['Account_Length','Area_Code','Phone_No','No_Vmail_Messages','Total_Day_minutes','Total_Day_Calls',
                    'Total_Day_charge','Total_Night_Calls','Total_Night_Charge','Total_Intl_Minutes','Total_Intl_Calls',
                    'Total_Intl_Charge','No_CS_Calls','State_cat','Voice_Mail_Plan_no','Voice_Mail_Plan_yes',
                    'International_Plan_no','International_Plan_yes','Churn']

In [10]:
##  Normalize data.
churn_norm = churnTrain[['Account_Length','Area_Code','Phone_No','No_Vmail_Messages','Total_Day_minutes','Total_Day_Calls',
                         'Total_Day_charge','Total_Night_Calls','Total_Night_Charge','Total_Intl_Minutes','Total_Intl_Calls',
                         'Total_Intl_Charge','No_CS_Calls','State_cat','Voice_Mail_Plan_ no','Voice_Mail_Plan_ yes',
                         'International_Plan_ no','International_Plan_ yes']].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
churn_norm.sample(n=5)

,Account_Length,Area_Code,Phone_No,No_Vmail_Messages,Total_Day_minutes,Total_Day_Calls,Total_Day_charge,Total_Night_Calls,Total_Night_Charge,Total_Intl_Minutes,Total_Intl_Calls,Total_Intl_Charge,No_CS_Calls,State_cat,Voice_Mail_Plan_ no,Voice_Mail_Plan_ yes,International_Plan_ no,International_Plan_ yes
388,0.347107,0.068627,0.347544,0.000000,0.290764,0.575758,0.290744,0.507042,0.336521,0.535,0.15,0.535185,0.111111,0.96,1.0,0.0,1.0,0.0
2128,0.297521,0.000000,0.707978,0.509804,0.374002,0.593939,0.373910,0.443662,0.534369,0.510,0.30,0.509259,0.222222,0.52,0.0,1.0,1.0,0.0
1995,0.483471,1.000000,0.563108,0.686275,0.730616,0.721212,0.730550,0.683099,0.517633,0.585,0.05,0.585185,0.111111,0.66,0.0,1.0,1.0,0.0
199,0.256198,0.000000,0.756829,0.000000,0.236602,0.387879,0.236586,0.394366,0.598924,0.650,0.15,0.650000,0.000000,0.14,1.0,0.0,1.0,0.0
547,0.388430,1.000000,0.534411,0.803922,0.389966,0.551515,0.390007,0.239437,0.297669,0.515,0.45,0.514815,0.555556,0.92,0.0,1.0,0.0,1.0


In [11]:
target = churnTrain['Churn']

In [12]:
df = pd.concat([target,churn_norm], axis=1)
df.sample(5)

,Churn,Account_Length,Area_Code,Phone_No,No_Vmail_Messages,Total_Day_minutes,Total_Day_Calls,Total_Day_charge,Total_Night_Calls,Total_Night_Charge,Total_Intl_Minutes,Total_Intl_Calls,Total_Intl_Charge,No_CS_Calls,State_cat,Voice_Mail_Plan_ no,Voice_Mail_Plan_ yes,International_Plan_ no,International_Plan_ yes
1157,0,0.475207,0.000000,0.957124,0.0,0.417332,0.745455,0.417337,0.485915,0.509863,0.390,0.25,0.390741,0.111111,0.18,1.0,0.0,1.0,0.0
225,0,0.264463,0.068627,0.645016,0.0,0.608324,0.672727,0.608317,0.633803,0.610281,0.135,0.20,0.135185,0.111111,0.30,1.0,0.0,1.0,0.0
841,0,0.314050,0.068627,0.771915,0.0,0.681870,0.690909,0.681757,0.338028,0.370592,0.515,0.10,0.514815,0.555556,0.66,1.0,0.0,1.0,0.0
1124,0,0.685950,0.068627,0.861853,0.0,0.474344,0.515152,0.474346,0.436620,0.554094,0.495,0.25,0.494444,0.111111,0.68,1.0,0.0,1.0,0.0
2069,1,0.214876,0.000000,0.768915,0.0,0.744584,0.721212,0.744467,0.556338,0.411237,0.490,0.10,0.490741,0.000000,0.74,1.0,0.0,1.0,0.0


#### Mark some of the data for testing purpose

In [13]:
train_test_per = 70/100.0
df['train'] = np.random.rand(len(df)) < train_test_per
df.sample(5)

,Churn,Account_Length,Area_Code,Phone_No,No_Vmail_Messages,Total_Day_minutes,Total_Day_Calls,Total_Day_charge,Total_Night_Calls,Total_Night_Charge,Total_Intl_Minutes,Total_Intl_Calls,Total_Intl_Charge,No_CS_Calls,State_cat,Voice_Mail_Plan_ no,Voice_Mail_Plan_ yes,International_Plan_ no,International_Plan_ yes,train
2431,0,0.752066,0.068627,0.471300,0.607843,0.488027,0.630303,0.487928,0.366197,0.466228,0.510,0.20,0.509259,0.111111,0.82,0.0,1.0,1.0,0.0,True
2275,0,0.429752,0.068627,0.514630,0.000000,0.651083,0.606061,0.651073,0.753521,0.597729,0.385,0.35,0.385185,0.000000,0.66,1.0,0.0,0.0,1.0,True
2634,0,0.417355,0.068627,0.104329,0.000000,0.350912,0.642424,0.350939,0.295775,0.595935,0.630,0.35,0.629630,0.222222,0.02,1.0,0.0,1.0,0.0,True
1026,0,0.272727,0.068627,0.121623,0.549020,0.270810,0.569697,0.270791,0.570423,0.367005,0.500,0.10,0.500000,0.222222,0.02,0.0,1.0,1.0,0.0,True
3190,1,0.421488,0.068627,0.203288,0.000000,0.498005,0.915152,0.497988,0.535211,0.390317,0.790,0.15,0.790741,0.666667,0.26,1.0,0.0,1.0,0.0,True


In [14]:
train = df[df.train==1]
train = train.drop('train', axis=1).sample(frac=1)
train.sample(5)

,Churn,Account_Length,Area_Code,Phone_No,No_Vmail_Messages,Total_Day_minutes,Total_Day_Calls,Total_Day_charge,Total_Night_Calls,Total_Night_Charge,Total_Intl_Minutes,Total_Intl_Calls,Total_Intl_Charge,No_CS_Calls,State_cat,Voice_Mail_Plan_ no,Voice_Mail_Plan_ yes,International_Plan_ no,International_Plan_ yes
561,0,0.214876,0.0,0.994777,0.352941,0.418472,0.648485,0.418511,0.683099,0.418410,0.360,0.35,0.359259,0.000000,0.78,0.0,1.0,1.0,0.0
140,0,0.450413,1.0,0.199499,0.000000,0.423318,0.696970,0.423374,0.556338,0.458458,0.620,0.15,0.620370,0.111111,0.16,1.0,0.0,1.0,0.0
2441,0,0.454545,0.0,0.777566,0.254902,0.550456,0.630303,0.550469,0.429577,0.549313,0.605,0.20,0.605556,0.111111,0.22,0.0,1.0,1.0,0.0
1349,0,0.289256,1.0,0.793503,0.431373,0.403079,0.648485,0.403085,0.464789,0.529588,0.280,0.15,0.279630,0.222222,0.34,0.0,1.0,1.0,0.0
473,1,0.293388,1.0,0.954831,0.647059,0.275371,0.357576,0.275319,0.591549,0.377167,0.310,0.20,0.309259,0.444444,0.98,0.0,1.0,1.0,0.0


In [15]:
test = df[df.train==0]
test = test.drop('train', axis=1)
test.sample(5)

,Churn,Account_Length,Area_Code,Phone_No,No_Vmail_Messages,Total_Day_minutes,Total_Day_Calls,Total_Day_charge,Total_Night_Calls,Total_Night_Charge,Total_Intl_Minutes,Total_Intl_Calls,Total_Intl_Charge,No_CS_Calls,State_cat,Voice_Mail_Plan_ no,Voice_Mail_Plan_ yes,International_Plan_ no,International_Plan_ yes
2241,0,0.227273,0.068627,0.088380,0.568627,0.107469,0.696970,0.107478,0.478873,0.547519,0.245,0.15,0.244444,0.111111,0.94,0.0,1.0,1.0,0.0
1011,0,0.041322,0.068627,0.775459,0.000000,0.408780,0.787879,0.408786,0.471831,0.459653,0.485,0.30,0.485185,0.222222,0.68,1.0,0.0,1.0,0.0
2032,0,0.619835,0.068627,0.753757,0.000000,0.338940,0.775758,0.338867,0.197183,0.462044,0.660,0.50,0.659259,0.222222,0.62,1.0,0.0,1.0,0.0
2513,0,0.351240,1.000000,0.626738,0.000000,0.201539,0.757576,0.201543,0.521127,0.407651,0.000,0.00,0.000000,0.222222,0.68,1.0,0.0,1.0,0.0
2389,1,0.533058,0.068627,0.723066,0.000000,0.606613,0.618182,0.606640,0.507042,0.395099,0.530,0.20,0.529630,0.000000,0.80,1.0,0.0,1.0,0.0


In [16]:
X = train.values[:,:18]
X[:5]

array([[0.        , 0.29338843, 0.        , 0.5927484 , 0.        ,
        0.51282782, 0.68484848, 0.51274313, 0.32394366, 0.3903168 ,
        0.49      , 0.35      , 0.49074074, 0.22222222, 0.96      ,
        1.        , 0.        , 1.        ],
       [0.        , 0.4214876 , 0.06862745, 0.55829769, 0.        ,
        0.45239453, 0.54545455, 0.45238095, 0.3028169 , 0.42498506,
        0.525     , 0.2       , 0.52592593, 0.11111111, 0.18      ,
        1.        , 0.        , 1.        ],
       [0.        , 0.54545455, 0.        , 0.60527935, 0.76470588,
        0.68386545, 0.64848485, 0.68376928, 0.36619718, 0.2839211 ,
        0.335     , 0.15      , 0.33518519, 0.55555556, 0.9       ,
        0.        , 1.        , 1.        ],
       [0.        , 0.49173554, 0.06862745, 0.98254469, 0.        ,
        0.37542759, 0.6       , 0.37541918, 0.58450704, 0.47878063,
        0.535     , 0.15      , 0.53518519, 0.11111111, 0.5       ,
        1.        , 0.        , 1.        ],
    

In [17]:
y = np.array([target[int(x)] for x in train.values[:,4:5]])
y[:5]

array([0, 0, 0, 0, 0], dtype=int8)

#### Model neural network by Keras and Tensorflow backend

In [27]:
## Input layer
model = Sequential()
model.add(Dense(50, input_dim=18, activation='relu'))

## Hidden Layers
model.add(layers.Dropout(0.3, noise_shape=None, seed=None))
model.add(Dense(50, activation='relu'))
model.add(layers.Dropout(0.2, noise_shape=None, seed=None))
model.add(Dense(50, activation='relu'))
model.add(layers.Dropout(0.1, noise_shape=None, seed=None))
model.add(Dense(50, activation='relu'))

##  Output Layer
model.add(Dense(1,activation='sigmoid'))

In [28]:
## Compile Model
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [29]:
model.fit(X,y, epochs=1000, batch_size=10)

Epoch 1/1000
2289/2289 [==============================] - 1s 284us/step - loss: 0.0430 - acc: 0.9956
Epoch 2/1000
2289/2289 [==============================] - 0s 124us/step - loss: 8.5845e-05 - acc: 1.0000
Epoch 3/1000
2289/2289 [==============================] - 0s 110us/step - loss: 3.8584e-05 - acc: 1.0000
Epoch 4/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0575e-05 - acc: 1.0000
Epoch 5/1000
2289/2289 [==============================] - 0s 121us/step - loss: 5.6944e-06 - acc: 1.0000
Epoch 6/1000
2289/2289 [==============================] - 0s 112us/step - loss: 2.3960e-06 - acc: 1.0000
Epoch 7/1000
2289/2289 [==============================] - 0s 123us/step - loss: 4.0290e-06 - acc: 1.0000
Epoch 8/1000
2289/2289 [==============================] - 0s 106us/step - loss: 1.9626e-06 - acc: 1.0000
Epoch 9/1000
2289/2289 [==============================] - 0s 117us/step - loss: 1.6516e-06 - acc: 1.0000
Epoch 10/1000
2289/2289 [==============================] - 

2289/2289 [==============================] - 0s 134us/step - loss: 1.0257e-07 - acc: 1.0000
Epoch 79/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 80/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 81/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 82/1000
2289/2289 [==============================] - 0s 110us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 83/1000
2289/2289 [==============================] - 0s 107us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 84/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 85/1000
2289/2289 [==============================] - 0s 119us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 86/1000
2289/2289 [==============================] - 0s 115us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 87/1000
2289/2289 [==============================] - 0

2289/2289 [==============================] - 0s 108us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 155/1000
2289/2289 [==============================] - 0s 102us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 156/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 157/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 158/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 159/1000
2289/2289 [==============================] - 0s 114us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 160/1000
2289/2289 [==============================] - 0s 110us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 161/1000
2289/2289 [==============================] - 0s 110us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 162/1000
2289/2289 [==============================] - 0s 119us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 163/1000
2289/2289 [==========================

2289/2289 [==============================] - 0s 103us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 231/1000
2289/2289 [==============================] - 0s 106us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 232/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0010e-07 - acc: 1.0000
Epoch 233/1000
2289/2289 [==============================] - 0s 107us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 234/1000
2289/2289 [==============================] - 0s 112us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 235/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 236/1000
2289/2289 [==============================] - 0s 113us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 237/1000
2289/2289 [==============================] - 0s 112us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 238/1000
2289/2289 [==============================] - 0s 110us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 239/1000
2289/2289 [==========================

2289/2289 [==============================] - 0s 114us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 307/1000
2289/2289 [==============================] - 0s 100us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 308/1000
2289/2289 [==============================] - 0s 106us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 309/1000
2289/2289 [==============================] - 0s 107us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 310/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 311/1000
2289/2289 [==============================] - 0s 104us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 312/1000
2289/2289 [==============================] - 0s 107us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 313/1000
2289/2289 [==============================] - 0s 110us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 314/1000
2289/2289 [==============================] - 0s 113us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 315/1000
2289/2289 [==========================

...

2289/2289 [==============================] - 0s 114us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 687/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 688/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 689/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 690/1000
2289/2289 [==============================] - 0s 114us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 691/1000
2289/2289 [==============================] - 0s 108us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 692/1000
2289/2289 [==============================] - 0s 97us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 693/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 694/1000
2289/2289 [==============================] - 0s 102us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 695/1000
2289/2289 [===========================

2289/2289 [==============================] - 0s 113us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 763/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 764/1000
2289/2289 [==============================] - 0s 110us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 765/1000
2289/2289 [==============================] - 0s 118us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 766/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 767/1000
2289/2289 [==============================] - 0s 106us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 768/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 769/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 770/1000
2289/2289 [==============================] - 0s 104us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 771/1000
2289/2289 [==========================

2289/2289 [==============================] - 0s 120us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 839/1000
2289/2289 [==============================] - 0s 101us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 840/1000
2289/2289 [==============================] - 0s 114us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 841/1000
2289/2289 [==============================] - 0s 105us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 842/1000
2289/2289 [==============================] - 0s 117us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 843/1000
2289/2289 [==============================] - 0s 108us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 844/1000
2289/2289 [==============================] - 0s 104us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 845/1000
2289/2289 [==============================] - 0s 113us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 846/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 847/1000
2289/2289 [==========================

2289/2289 [==============================] - 0s 117us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 915/1000
2289/2289 [==============================] - 0s 108us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 916/1000
2289/2289 [==============================] - 0s 102us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 917/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 918/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 919/1000
2289/2289 [==============================] - 0s 102us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 920/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 921/1000
2289/2289 [==============================] - 0s 106us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 922/1000
2289/2289 [==============================] - 0s 102us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 923/1000
2289/2289 [==========================

2289/2289 [==============================] - 0s 115us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 991/1000
2289/2289 [==============================] - 0s 108us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 992/1000
2289/2289 [==============================] - 0s 111us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 993/1000
2289/2289 [==============================] - 0s 109us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 994/1000
2289/2289 [==============================] - 0s 101us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 995/1000
2289/2289 [==============================] - 0s 106us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 996/1000
2289/2289 [==============================] - 0s 115us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 997/1000
2289/2289 [==============================] - 0s 121us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 998/1000
2289/2289 [==============================] - 0s 110us/step - loss: 1.0000e-07 - acc: 1.0000
Epoch 999/1000
2289/2289 [==========================

In [26]:
scores = model.evaluate(X,y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

2289/2289 [==============================] - 0s 11us/step

acc: 100.00%
